In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
from glob import glob

In [ ]:
sns.set(style='whitegrid', palette='muted', font='serif')

In [ ]:
dfs = []
for fn in glob('../learningsys-2018-gpu-mux/p3-8xlarge-random-placement/**/*.pq', recursive=True):
    df = pd.read_parquet(fn)
    approach, model, proc, i  = fn.split("/")[4:]
    df['approach'] = approach
    df['model'] = model
    df['n_procs'] = int(proc)
    df['proc'] = int(i.split('.')[0])
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
# dfs = []
# for fn in glob('learningsys-2018-gpu-mux/**/*.pq', recursive=True):
#     df = pd.read_parquet(fn)
#     _, _, approach, model, proc, i  = fn.split("/")
#     df['approach'] = approach
#     df['model'] = model
#     df['n_procs'] = int(proc)
#     df['proc'] = int(i.split('.')[0])
#     dfs.append(df)
# df = pd.concat(dfs)

In [ ]:
anomoly = df.groupby(
    ['approach', 'model', 'n_procs', 'proc']
)[['duration_ms']].mean().reset_index().groupby(
    ['approach', 'model', 'n_procs'])[['duration_ms']].apply(lambda s: s.max() - s.min()).reset_index()

In [ ]:
anomoly = anomoly[(anomoly['duration_ms']>0.1) & (anomoly['approach'] == 'mps')]

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
sns.pointplot(x='n_procs', y='duration_ms', hue='model', data=anomoly)
plt.axhline(0.1, label='0.1 lower bound')
plt.legend()
plt.xlabel('Number of Replicas')
plt.ylabel('Latency (ms)')
plt.title('Max difference between processes')

plt.subplot(1,3, 2)
res152_5_procs = df[
    (df['model'] == 'mobilenet') & 
    (df['n_procs'] == 16) & 
    (df['approach'] == 'mps')][['duration_ms', 'proc']]
grouped = res152_5_procs.groupby('proc')[['duration_ms']].rolling(20).mean().dropna().reset_index()
grouped['level_1'] = grouped['level_1'].astype(int)
sns.lineplot(x='level_1', y='duration_ms', hue='proc', data=grouped)
plt.xlabel('Query IDs')
plt.ylabel('Latency (ms)')
plt.title('Example: 16 Processes')


plt.subplot(1,3,3)
res152_5_procs = df[
    (df['model'] == 'mobilenet') & 
    (df['n_procs'] == 15) & 
    (df['approach'] == 'mps')][['duration_ms', 'proc']]
grouped = res152_5_procs.groupby('proc')[['duration_ms']].rolling(20).mean().dropna().reset_index()
grouped['level_1'] = grouped['level_1'].astype(int)
sns.lineplot(x='level_1', y='duration_ms', hue='proc', data=grouped)
plt.xlabel('Query IDs')
plt.ylabel('Latency (ms)')
plt.title('Example: 15 Processes')

plt.savefig('anomoly.png', dpi=300)

In [ ]:
grouped.groupby('proc')['duration_ms'].mean().sort_values()

In [ ]:
res152_5_procs

In [ ]:
for _, row in anomoly.iterrows():
    print(f"{row['approach']}-{row['model']}-{row['n_procs']}")

In [ ]:
len(anomoly)